In [27]:
import requests
import pandas as pd
import numpy as np
from elasticsearch import Elasticsearch
import os
from datetime import datetime as dt
import os.path
from os import path

In [2]:
ls

 CKAN_to_elastic_encrypted.ipynb  'kibana_migrate_index (1) (1).ipynb'


In [4]:
url_ckan='http://datos.gob.cl/api/3/action/package_list'
url_package='http://datos.gob.cl/api/3/action/package_show?id='
url_package_search='http://datos.gob.cl/api/3/action/package_search?'

In [8]:
#Listas de paquetes ckan (con sus ID)
ckan_packages=requests.get(url_ckan) #consultar api
ckan_packages=ckan_packages.json() #convertir respuesta a json

In [10]:
ckan_packages

{'help': 'http://datos.gob.cl/api/3/action/help_show?name=package_list',
 'success': True,
 'result': ['10000',
  '10002',
  '10006',
  '10007',
  '10010',
  '10011',
  '10014',
  '10029',
  '10037',
  '10043',
  '10049',
  '10054',
  '10063',
  '10069',
  '10071',
  '10078',
  '10080',
  '10101',
  '10103',
  '10113',
  '10126',
  '10127',
  '10138',
  '10145',
  '10146',
  '10154',
  '10201',
  '10203',
  '10206',
  '10208',
  '10223',
  '10232',
  '10272',
  '10279',
  '10284',
  '10286',
  '10288',
  '1030',
  '1036',
  '10385',
  '1044',
  '10451',
  '10453',
  '1047',
  '1048',
  '1049',
  '1050',
  '1051',
  '1052',
  '1053',
  '1054',
  '1055',
  '1056',
  '1057',
  '1058',
  '1059',
  '10616',
  '1063',
  '1065',
  '10657',
  '10661',
  '10662',
  '10664',
  '10667',
  '10670',
  '10671',
  '10674',
  '10675',
  '10678',
  '10687',
  '10713',
  '1073',
  '10740',
  '1080',
  '10807',
  '10808',
  '10809',
  '1081',
  '1082',
  '10825',
  '10826',
  '10827',
  '10828',
  '10829

In [28]:
#OBTENER TODOS LOS PAQUETES DE LA BASE DE DATOS CKAN
rows=str(1000) #maximo numero de resultados por consulta (maximo permitido es 1000)
packs=[] #contenedor de paquetes retornados
for s in range(0,4):
    start=str(s*int(rows)) #paginacion
    packages=requests.get(url_package_search+'rows='+rows+'&'+'start='+start)
    packages=packages.json()
    packs.extend(packages['result']['results'])

JSONDecodeError: Expecting value: line 2 column 5 (char 5)

In [6]:
recursos=[]
#recorrer recursos y ordenarlos en una lista (con ciertos atributos)
for p in packs:
    for r in p['resources']:
        d={} #contenedor para valores
        d['fecha']=r['last_modified'] #ultima modificacion
        d['formato']=r['format'] #formato
        if 'organization' in p.keys(): #si es que existe info de organizacion
            if p['organization']!=None:
                d['inst.']=p['organization']['title'] #si es que existe info del titulo incluirlo
            else:
                d['inst.']=None
        else:
            d['inst.']=None
        recursos.append(d)
recursos=pd.DataFrame(recursos) #convertir a df
recursos['fecha']=pd.to_datetime(recursos['fecha']) #convertir la fecha a formato solo con la fecha (sin la hora)
#desagregar en dia, mes, año
recursos['dia']=recursos['fecha'].dt.date
recursos['year']=recursos['fecha'].dt.year
recursos['mes']=recursos['fecha'].dt.month

In [94]:
#guardar paso en csv
path='/data/cache/ckan/state_datos_gob'
now = dt.now()
date_time = now.strftime("_%m-%d-%Y_%H-%M-%S")

In [95]:
recursos.to_csv(path+date_time+'.csv')

In [15]:
#RUTA PARA RECUPERAR PASO GUARDADO EN CSV
#ls /data/cache/ckan/state_datos_gob*

In [14]:
#RECUPERAR PASO GUARDADO EN CSV
#recursos=pd.read_csv('/data/cache/ckan/state_datos_gob_03-31-2020_05-05-15.csv',index_col=0)

In [16]:
data=recursos.to_dict(orient='records')

In [17]:
bulk_data=[]
for idx in range(0,len(data)):
    bulk_data.append({"index":{ "_index" : "ckan", "_type" : "dataset", "_id" : str(idx) }})
    bulk_data.append(data[idx])

In [18]:
ES_HOST = "https://:9200"
INDEX_NAME = 'ckan'

In [19]:
#variables de entorno
args={}
args['elastic_user']='elastic'
args['elastic_psswd']=os.environ['ELASTIC_PASSWD']

In [20]:
args['elastic_psswd']

'fjhumeres'

In [21]:
es = Elasticsearch(["https://"], http_auth=(args['elastic_user'], args['elastic_psswd']))

In [30]:
es.ping()

True

In [ ]:
def elastic_prepare_pattern(self,rw=True):
    docs=self.docs.copy()
    #convertir nulos
    docs=self.nulls(docs)
    #generar ids para elastic
    if self.has_attribute('generateID_params'): #nativos,a partir de algún campo '_id'
        docs=self.generate_id(docs,**self.generateID_params)
    if self.has_attribute('generateID_params_custom'): #a partir de concatenar varios campos
        docs=self.custom_id(docs,**self.generateID_params_custom)
    #patrón de retornar documentos o nada (y guardar como atributo de la instancia), según parametro 'rw' (read-write)
        
    return self.read_write_pattern(docs,rw=rw) 

In [31]:
if es.indices.exists(INDEX_NAME):
    print("deleting '%s' index..." % (INDEX_NAME))
    res = es.indices.delete(index = INDEX_NAME)
    print(" response: '%s'" % (res))
# since we are running locally, use one shard and no replicas
request_body = {
    "settings" : {
        "number_of_shards": 1,
        "number_of_replicas": 0
    }
}
print("creating '%s' index..." % (INDEX_NAME))
res = es.indices.create(index = INDEX_NAME, body = request_body)
print(" response: '%s'" % (res))

deleting 'ckan' index...
 response: '{'acknowledged': True}'
creating 'ckan' index...
 response: '{'acknowledged': True, 'shards_acknowledged': True, 'index': 'ckan'}'


In [32]:
# bulk index the data
print("bulk indexing...")
res = es.bulk(index = INDEX_NAME, body = bulk_data, refresh = True)

bulk indexing...


In [33]:
# sanity check
res = es.search(index = INDEX_NAME, size=2, body={"query": {"match_all": {}}})
print(" response: '%s'" % (res))

 response: '{'took': 2, 'timed_out': False, '_shards': {'total': 1, 'successful': 1, 'skipped': 0, 'failed': 0}, 'hits': {'total': {'value': 6301, 'relation': 'eq'}, 'max_score': 1.0, 'hits': [{'_index': 'ckan', '_type': 'dataset', '_id': '0', '_score': 1.0, '_source': {'fecha': '2020-03-25 21:14:45.575015', 'formato': 'XLSX', 'inst.': 'Subsecretaría de Transporte', 'dia': '2020-03-25', 'year': 2020.0, 'mes': 3.0}}, {'_index': 'ckan', '_type': 'dataset', '_id': '1', '_score': 1.0, '_source': {'fecha': '2020-03-25 21:12:02.566500', 'formato': 'XLSX', 'inst.': 'Subsecretaría de Transporte', 'dia': '2020-03-25', 'year': 2020.0, 'mes': 3.0}}]}}'


In [34]:
#contar documentos
def count_documents(es,INDEX_NAME):
    es.indices.refresh(INDEX_NAME)
    res=es.cat.count(INDEX_NAME, params={"format": "json"})
    print(res)
        
count_documents(es,INDEX_NAME)

[{'epoch': '1585692683', 'timestamp': '22:11:23', 'count': '6301'}]
